<a href="https://colab.research.google.com/github/ksushalykova/LiveCorpus/blob/master/%D0%9B%D1%8B%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9A%D1%81%D0%B5%D0%BD%D0%B8%D1%8F_NNLP_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 4

В домашнем задании нужно обучить модель для ответов на вопросы. Будем использовать датасет SQUAD, включающий вопросы, контекст и ответы.

Цель задания: поэкспериментировать с генерацией и проанализировать результаты. В качестве модели можно выбрать модель на основе декодера трансформера или модель с архитектурой Encoder-Decoder.

Баллы за ДЗ:



*   Предобработка и токенизатор - 2 балл
*   Загрузка и обучение модели - 3 балла
*   Инференс и эксперименты - 3 балла
*   Отчёт - 2 балла
* Бонус - 5 баллов


## Загузка датасета

In [ ]:
!pip install transformers datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 563.7 kB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset('squad')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
ds['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
# creating a smaller dataset

small_ds = ds.filter(lambda example, index: index % 20 == 0, with_indices=True)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
small_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4380
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 529
    })
})

## Предобработка и токенизатор

Далее нужно подготовить и токенизировать данные для обучения модели.

В зависимости от выбора модели формат представления данных будет различаться.

* Для генеративной модели на основе декодера (GPT) нужно подготовить промпт в качестве условия для генерации. Например, данные для одного вопроса могут выглядеть так: '<context> Question: <question> Answer: <answer>'. С форматом промпта можно экспеиментировать.

* Для Encoder-Decoder модели (например, T5) нужно отдельно токенизировать входные данные (контекст и вопрос) и таргет (ответ). В начале инпута возможно нужно будет указать префикс задания для модели.



In [ ]:
# TODO: download a tokenizer and  preprocess our data


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=301b441bb5bf016a737eb13b8fcf2f0e657178e8368ee862f117c553764f9db7
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
import transformers

In [ ]:
# GPT

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, AutoModelWithLMHead
from transformers import AutoModel, AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("gpt2")
#model = AutoModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model = AutoModelWithLMHead.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# add the EOS token as PAD token to avoid warnings
#model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
# 1 (prompt in text files format 'Context: Question: Answer: ' -- success)

In [ ]:
# каждый из таких вариантов (и для gpt, и для t5) нужно запускать отдельно от других в порядке предобработка/токенизатор -> модель

In [ ]:
import re

def build_text_files(data_set, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_set:
        title = str(texts['title']).strip()
        context = str(texts['context']).strip()
        full_context = title + context
        question = str(texts['question']).strip()
        answer = str(texts['answers']['text'][0]).strip()
        data += 'Context: ' + full_context + ' ' + 'Question: ' + question + ' ' + 'Answer: ' + answer + ' '
    f.write(data)


In [ ]:
build_text_files(small_ds['train'],'train_dataset.txt')
build_text_files(small_ds['validation'],'test_dataset.txt')

In [ ]:
train_path = 'train_dataset.txt'
val_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=val_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, val_path, tokenizer)

In [ ]:
train_dataset[0]

tensor([21947,    25,  2059,    62,  1659,    62,  3673,   260,    62,    35,
          480, 19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,
         2095,    13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,
          318,   257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,
          287,  2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,
          318,   257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,
          351,   262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,
         1911,  7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,
          286,   262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,
          318,   262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,
          290, 14580,    13,   632,   318,   257, 30069,   286,   262,  7128,
        33955,   379,   406,   454,  8906,    11,  4881,   810,   262,  5283,
         5335,  1128,  7241,   306,  4120,   284,  9281,  6206])

In [ ]:
tokenizer.convert_ids_to_tokens(train_dataset[0])

['Context',
 ':',
 'ĠUniversity',
 '_',
 'of',
 '_',
 'Not',
 're',
 '_',
 'D',
 'ame',
 'Arch',
 'itect',
 'urally',
 ',',
 'Ġthe',
 'Ġschool',
 'Ġhas',
 'Ġa',
 'ĠCatholic',
 'Ġcharacter',
 '.',
 'ĠAt',
 'op',
 'Ġthe',
 'ĠMain',
 'ĠBuilding',
 "'s",
 'Ġgold',
 'Ġdome',
 'Ġis',
 'Ġa',
 'Ġgolden',
 'Ġstatue',
 'Ġof',
 'Ġthe',
 'ĠVirgin',
 'ĠMary',
 '.',
 'ĠImmediately',
 'Ġin',
 'Ġfront',
 'Ġof',
 'Ġthe',
 'ĠMain',
 'ĠBuilding',
 'Ġand',
 'Ġfacing',
 'Ġit',
 ',',
 'Ġis',
 'Ġa',
 'Ġcopper',
 'Ġstatue',
 'Ġof',
 'ĠChrist',
 'Ġwith',
 'Ġarms',
 'Ġup',
 'raised',
 'Ġwith',
 'Ġthe',
 'Ġlegend',
 'Ġ"',
 'Ven',
 'ite',
 'ĠAd',
 'ĠMe',
 'ĠOm',
 'nes',
 '".',
 'ĠNext',
 'Ġto',
 'Ġthe',
 'ĠMain',
 'ĠBuilding',
 'Ġis',
 'Ġthe',
 'ĠBasil',
 'ica',
 'Ġof',
 'Ġthe',
 'ĠSacred',
 'ĠHeart',
 '.',
 'ĠImmediately',
 'Ġbehind',
 'Ġthe',
 'Ġbasil',
 'ica',
 'Ġis',
 'Ġthe',
 'ĠGro',
 'tto',
 ',',
 'Ġa',
 'ĠMarian',
 'Ġplace',
 'Ġof',
 'Ġprayer',
 'Ġand',
 'Ġreflection',
 '.',
 'ĠIt',
 'Ġis',
 'Ġa',
 'Ġrepli

In [ ]:
# 2 (attempts to make prompt in other different formats -- fail)

In [ ]:
# попытка сделать attention mask = 0 для answer части промпта, для context и question частей attention mask = 1

def my_data_collator(text_str):
    encoded_results = tokenizer(text_str, padding=True, truncation=True, return_tensors='pt', return_attention_mask=True)
    encoded_results['attention_mask'] = set_my_attention_mask(encoded_results) #function to set attention mask to 0 on tokens in the answer:... part of text_str
    label_ids = get_my_label_str(encoded_results['input_ids']) #function to return list of token ids for answer:... part of text_str

    batch = {}
    batch['input_ids'] = encoded_results['input_ids']
    batch['past'] = None
    batch['attention_mask'] = encoded_results['attention_mask']
    batch['position_ids'] = None
    batch['head_mask'] = None
    batch['inputs_embeds'] = None
    batch['labels'] = label_ids
    batch['use_cache'] = True
    return batch

In [ ]:
# attempt to make prompt in format of list
import re

def build_files(data_set):
    #f = open(dest_path, 'w')
    data_all = []
    for texts in data_set:
        data = ''
        title = str(texts['title']).strip()
        context = str(texts['context']).strip()
        full_context = title + context
        question = str(texts['question']).strip()
        answer = str(texts['answers']['text'][0]).strip()
        data += 'Context: ' + full_context + ' ' + 'Question: ' + question + ' ' + 'Answer: ' + answer + ' '
        data_all.append(data)
    #f.write(data_all)
    return data_all

In [ ]:
train_dataset = build_files(small_ds['train'])
val_dataset = build_files(small_ds['validation'])
print(train_dataset)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_dataset[0]

'Context: University_of_Notre_DameArchitecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? Answer: Saint Bernadette Soubirous '

In [ ]:

build_files(small_ds['train'],'train_dataset.txt')
build_files(small_ds['validation'],'test_dataset.txt')

TypeError: ignored

In [ ]:
# attempt to make prompt in format of dataset

max_input_length = 512
#max_target_length = 128

def preprocessing(examples):
    inf = []
    #print(examples)

    answ = []
    #print(examples['answers'])
    for s in examples['answers']:
        #print(s['text'][0])
        answ.append(s['text'][0])

    for i in range(len(examples['context'])):
        #print(examples['context'][i])
        context = ''
        context += 'Context: ' + examples['context'][i]
        question = ''
        question += 'Question: ' + examples['question'][i]
        answer = ''
        answer += 'Answer: ' + answ[i]
        info = ''
        info += context + ' ' + question + ' ' + answer
        #print(info)
        inf.append(info)
    inputs = [doc for doc in inf]
    #print(inputs)
    #tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token = tokenizer.eos_token
    model_inputs = tokenizer(text = inputs, max_length=max_input_length, truncation=True, padding = 'max_length')

    return model_inputs

In [ ]:
# attempt to add target to prompt

max_input_length = 300
max_target_length = 20


def preprocessing(examples):
    #input_data = examples['context'] + examples['question']
    inf = []
    #print(examples)
    for i in range(len(examples['context'])):
        #print(examples['context'][i])
        question = ''
        question += 'Question: ' + examples['question'][i]
        context = ''
        context += 'Context: ' + examples['context'][i]
        answer = ''
        answer += 'Answer: ' + examples['answers'][i]['text'][0]
        info = ''
        info += context + ' ' + question + ' ' + answer
        #print(info)
        inf.append(info)
    inputs = [doc for doc in inf]
    #print(inputs)
    tokenizer.pad_token = tokenizer.eos_token
    model_inputs = tokenizer(text = inputs, max_length=max_input_length, truncation=True, padding = 'max_length')

    # Setup the tokenizer for targets
    answ = []
    #print(examples['answers'])
    for s in examples['answers']:
        #print(s['text'][0])
        answ.append(s['text'][0])
    labels = tokenizer(text_target=answ, max_length=max_target_length, truncation=True)


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocessing(small_ds['train'][:2])

{'input_ids': [[21947, 25, 17340, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774, 11819, 338, 3869, 29500, 318, 257, 10861, 15207, 286, 262, 5283, 5335, 13, 34528, 287, 2166, 286, 262, 8774, 11819, 290, 6476, 340, 11, 318, 257, 15317, 15207, 286, 1951, 351, 5101, 510, 49309, 351, 262, 8177, 366, 37522, 578, 1215, 2185, 16543, 2516, 1911, 7406, 284, 262, 8774, 11819, 318, 262, 32520, 3970, 286, 262, 17380, 8894, 13, 34528, 2157, 262, 37792, 3970, 318, 262, 10299, 33955, 11, 257, 37919, 1295, 286, 11443, 290, 14580, 13, 632, 318, 257, 30069, 286, 262, 7128, 33955, 379, 406, 454, 8906, 11, 4881, 810, 262, 5283, 5335, 1128, 7241, 306, 4120, 284, 9281, 6206, 324, 5857, 311, 12944, 343, 516, 287, 1248, 3365, 13, 1629, 262, 886, 286, 262, 1388, 3708, 357, 392, 287, 257, 1277, 1627, 326, 20417, 832, 513, 25827, 290, 262, 3561, 31390, 828, 318, 257, 2829, 11, 3660, 7815, 15207, 286, 5335, 13, 18233, 25, 1675, 4150, 750, 262, 5283, 5335, 7910, 1656, 287, 1248, 3365, 287, 40

In [ ]:
tokenized_datasets = small_ds.map(preprocessing, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
    num_rows: 4380
})

In [ ]:
# T5

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from datasets import load_metric

rouge_metric = load_metric("rouge")
meteor_metric = load_metric("meteor")

<ipython-input-10-f7268f9b390b>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# 1
# prefix = question-answering

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "question-answering: "
else:
    prefix = ""

In [ ]:
# 1.1
# input = prefix + context + question (in that order)

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    #input_data = examples['context'] + examples['question']
    inputs = [prefix + doc for doc in examples['context']]
    #print(inputs)
    model_inputs = tokenizer(text = inputs, text_pair = examples['question'], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    answ = []
    #print(examples['answers'])
    for s in examples['answers']:
        #print(s['text'][0])
        answ.append(s['text'][0])
    labels = tokenizer(text_target=answ, max_length=max_target_length, truncation=True)


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = small_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [ ]:
# 1.2
# input = prefix + question + context (in that order)

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    #input_data = examples['context'] + examples['question']
    inputs = [prefix + doc for doc in examples['question']]
    #print(inputs)
    model_inputs = tokenizer(text = inputs, text_pair = examples['context'], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    answ = []
    #print(examples['answers'])
    for s in examples['answers']:
        #print(s['text'][0])
        answ.append(s['text'][0])
    labels = tokenizer(text_target=answ, max_length=max_target_length, truncation=True)


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = small_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [ ]:
# 2
# prefix = text2text-generation, input = prefix + question + context

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "text2text-generation: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    #input_data = examples['context'] + examples['question']
    inf = []
    #print(examples)
    for i in range(len(examples['context'])):
        #print(examples['context'][i])
        question = ''
        question += 'question: ' + examples['question'][i]
        context = ''
        context += 'context: ' + examples['context'][i]
        info = ''
        info += question + ' ' + context
        #print(info)
        inf.append(info)
    inputs = [prefix + doc for doc in inf]
    #print(inputs)
    model_inputs = tokenizer(text = inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    answ = []
    #print(examples['answers'])
    for s in examples['answers']:
        #print(s['text'][0])
        answ.append(s['text'][0])
    labels = tokenizer(text_target=answ, max_length=max_target_length, truncation=True)


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(small_ds['train'][:2])

{'input_ids': [[822, 18, 3247, 3321, 53, 10, 304, 4068, 410, 8, 16823, 3790, 3, 18280, 2385, 16, 507, 3449, 16, 301, 1211, 1395, 1410, 58, 1, 30797, 120, 6, 8, 496, 65, 3, 9, 6502, 1848, 5, 71, 2916, 8, 5140, 5450, 31, 7, 2045, 22161, 19, 3, 9, 7069, 12647, 13, 8, 16823, 3790, 5, 3, 29167, 16, 851, 13, 8, 5140, 5450, 11, 5008, 34, 6, 19, 3, 9, 8658, 12647, 13, 2144, 28, 6026, 3, 76, 24266, 28, 8, 9503, 96, 553, 15, 7980, 1980, 1212, 13285, 1496, 1280, 3021, 12, 8, 5140, 5450, 19, 8, 23711, 2617, 13, 8, 3, 24756, 6219, 5, 3, 29167, 1187, 8, 20605, 2617, 19, 8, 8554, 17, 235, 6, 3, 9, 17535, 286, 13, 7029, 11, 9619, 5, 94, 19, 3, 9, 16455, 13, 8, 3, 3844, 17, 235, 44, 301, 1211, 1395, 6, 1410, 213, 8, 16823, 3790, 3, 28285, 26, 120, 4283, 12, 2788, 8942, 9, 26, 1954, 264, 8371, 8283, 16, 507, 3449, 5, 486, 8, 414, 13, 8, 711, 1262, 41, 232, 16, 3, 9, 1223, 689, 24, 1979, 7, 190, 220, 12647, 7, 11, 8, 2540, 10576, 15, 201, 19, 3, 9, 650, 6, 941, 3372, 12647, 13, 3790, 5, 1], [822, 18, 324

In [ ]:
tokenized_datasets = small_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

## Обучение модели

In [ ]:
# TODO: declare and train your model

model = ###

In [ ]:
# GPT

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [ ]:
# 1 (successful prompt attempt -- from text files)

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-qa", #? #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, #3 # number of training epochs
    per_device_train_batch_size=8, #32 # batch size for training
    per_device_eval_batch_size=8,  #64 # batch size for evaluation
    eval_steps = 40, #400 # Number of update steps between two evaluations.
    save_steps=80, #800 # after # steps model is saved
    warmup_steps=50, #500 # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    fp16=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.552500


TrainOutput(global_step=793, training_loss=3.512970394023881, metrics={'train_runtime': 190.5321, 'train_samples_per_second': 33.265, 'train_steps_per_second': 4.162, 'total_flos': 414017224704000.0, 'train_loss': 3.512970394023881, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.39218807220459,
 'eval_runtime': 4.2383,
 'eval_samples_per_second': 185.215,
 'eval_steps_per_second': 23.358,
 'epoch': 1.0}

In [ ]:
trainer.save_model()

In [ ]:
# 2 (attempts of learning for failed prompts)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import DefaultDataCollator

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-qa2", #? #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, #3 # number of training epochs
    per_device_train_batch_size=8, #32 # batch size for training
    per_device_eval_batch_size=16,  #64 # batch size for evaluation
    eval_steps = 40, #400 # Number of update steps between two evaluations.
    save_steps=80, # 800 # after # steps model is saved
    warmup_steps=50, # 500 # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    #fp16=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


TypeError: ignored

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
# T5

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "t5-small"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    result.update(meteor_metric.compute(predictions=decoded_preds, references=decoded_labels))
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# 1.1 (prefix = question-answering, input = prefix + context + question)

In [ ]:
batch_size = 32
#model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-qa1", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    #f"{model_name}-finetuned-qa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, #0.5
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,No log,0.583670,59.585800,36.526800,59.559000,59.590800,5.294900,0.466100
2,No log,0.530770,64.504400,39.520000,64.419400,64.422100,5.349700,0.504200


TrainOutput(global_step=274, training_loss=1.0154980290545164, metrics={'train_runtime': 170.3488, 'train_samples_per_second': 51.424, 'train_steps_per_second': 1.608, 'total_flos': 952937394733056.0, 'train_loss': 1.0154980290545164, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5307697057723999,
 'eval_rouge1': 64.5044,
 'eval_rouge2': 39.52,
 'eval_rougeL': 64.4194,
 'eval_rougeLsum': 64.4221,
 'eval_gen_len': 5.3497,
 'eval_meteor': 0.5042,
 'eval_runtime': 10.9666,
 'eval_samples_per_second': 48.238,
 'eval_steps_per_second': 1.55,
 'epoch': 2.0}

In [ ]:
# evaluate можно делать на отдельной тестовой выборке, но оставлю так

In [ ]:
trainer.save_model()

In [ ]:
# 1.2 (prefix = question-answering, input = prefix + question + context)

In [ ]:
batch_size = 32
#model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-qa2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    #f"{model_name}-finetuned-qa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, #0.5
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,No log,0.408245,74.425700,46.923200,74.458300,74.402000,5.397000,0.585900
2,No log,0.403558,74.926700,47.355800,74.934200,74.814800,5.387500,0.590400


TrainOutput(global_step=274, training_loss=0.4985582115006273, metrics={'train_runtime': 161.8952, 'train_samples_per_second': 54.109, 'train_steps_per_second': 1.692, 'total_flos': 952937394733056.0, 'train_loss': 0.4985582115006273, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4035579264163971,
 'eval_rouge1': 74.9267,
 'eval_rouge2': 47.3558,
 'eval_rougeL': 74.9342,
 'eval_rougeLsum': 74.8148,
 'eval_gen_len': 5.3875,
 'eval_meteor': 0.5904,
 'eval_runtime': 10.6494,
 'eval_samples_per_second': 49.674,
 'eval_steps_per_second': 1.596,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

In [ ]:
# 2 (prefix = text2text-generation, input = prefix + question + context)

In [ ]:
batch_size = 32
#model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-qa3", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    #f"{model_name}-finetuned-qa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, #0.5
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,No log,0.380720,77.142000,49.505500,77.009000,77.055000,5.610600,0.617800
2,No log,0.378048,76.973900,49.510700,76.799600,76.842600,5.555800,0.615500


TrainOutput(global_step=274, training_loss=0.3937299686626796, metrics={'train_runtime': 159.4447, 'train_samples_per_second': 54.941, 'train_steps_per_second': 1.718, 'total_flos': 959884235636736.0, 'train_loss': 0.3937299686626796, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3780481219291687,
 'eval_rouge1': 76.9739,
 'eval_rouge2': 49.5107,
 'eval_rougeL': 76.7996,
 'eval_rougeLsum': 76.8426,
 'eval_gen_len': 5.5558,
 'eval_meteor': 0.6155,
 'eval_runtime': 11.2113,
 'eval_samples_per_second': 47.184,
 'eval_steps_per_second': 1.516,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

## Инференс модели

Запустите модель на нескольких примерах с помощью пайплайна. Можно поэкспериментировать с декодированием, задать разные параметры генерации. Сравните результаты.

In [ ]:
# TODO: pipeline for inference


In [ ]:
# GPT

In [ ]:
# saved model pipeline

from transformers import pipeline

answ = pipeline('text-generation', model="./gpt2-qa", tokenizer=GPT2Tokenizer.from_pretrained("gpt2"))

In [ ]:
small_ds['validation'][2]

{'id': '56bf159b3aeaaa14008c9507',
 'title': 'Super_Bowl_50',
 'context': 'The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.',
 'question': 'What was the win/loss ratio in 2015 for the Carolina Panthers during their regular season?',
 'answers': {'text': ['15–1', '15–1', '15–1'], 'answer_start': [48, 48, 48]}}

In [ ]:
# example 1 (data1 and data2 are experiments on prompt -- with the word "Answer:" or without)

In [ ]:
data1 = 'Context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl. Question: What was the win/loss ratio in 2015 for the Carolina Panthers during their regular season? Answer: '
data2 = 'Context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl. Question: What was the win/loss ratio in 2015 for the Carolina Panthers during their regular season? '

In [ ]:
answ(data1)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 151, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': 'Context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl. Question: What was the win/loss ratio in 2015 for the Carolina Panthers during their regular season? Answer: \xa0'}]

In [ ]:
answ(data2)

Input length of input_ids is 149, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': 'Context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl. Question: What was the win/loss ratio in 2015 for the Carolina Panthers during their regular season? ________________________'}]

In [ ]:
# example 2

In [ ]:
data3 = 'Context: My name is Jack. Question: What is your name? Answer: '
data4 = 'Context: My name is Jack. Question: What is your name?'

In [ ]:
answ(data3)

[{'generated_text': "Context: My name is Jack. Question: What is your name? Answer: __________________ Context: DogIn 2012, the New York Dog Summit was launched. A series of dog training sessions began with the introduction of the New York Dog's First"}]

In [ ]:
answ(data4)

[{'generated_text': 'Context: My name is Jack. Question: What is your name? Answer: Jacky Context: James_GillespieIt started in May 1990, when it launched out of a self-sustaining satellite, the Space Shuttle Endeavour'}]

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("./gpt2-qa", from_pt=True, pad_token_id=tokenizer.eos_token_id)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
# example 1 (prompt with the word "Answer: ")

In [ ]:
input_ids = tokenizer.encode("Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: ", return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _________C_P_P_P_P_P_P_P-P-P-P-P-P-P-P


In [ ]:
# beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: __________ Context: Matt_CrusaderMatt Crusader was born in New Haven, Connecticut, on January 1, 1857. He was the


In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=3,
    no_repeat_ngram_size=2, #
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer:???? Context: Matthew_MasonMatthew Mason was born in New Haven, Connecticut, in 1842. He attended the University of Connecticut and Yale University,


In [ ]:
# sampling with temperature

tf.random.set_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7 # 1
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _______________ Context: Matt_Tranzaco_(torch)As the name implies, Matt's car was a relatively old-fashioned car, built


In [ ]:
# top k sampling

tf.random.set_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _______________ Context: Matt_TranquillaMany historians of Triton suggest that Tacitus's account of the events of the early Christian era has


In [ ]:
# top p sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: original: weekpage Context: MattCover story: Tony_VaughnTalk to Captain America during the months following the events of Avengers: Age of Ultron


In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=20,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _________C_G_T_L_S_P_E-2 Context: Matt_ToadThe most important part of the story of


In [ ]:
# sevaral variants of generation

tf.random.set_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _______________ Context: Matt_TranquillaFor many of his childhood friends and family, Matt's actions were thought of as a self-defeating
1: Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: _________________________________________Postscript: I did not find the original photo of Matt. I also found a video of the incident from the night Matt was killed,
2: Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: ʻDʻavᵛᵔᵖ Context: Matt_MacyMacaulay's new show, True Life, premiered on Fox


In [ ]:
# example 1 (prompt without the word "Answer: ")

In [ ]:
input_ids = tokenizer.encode("Context: Matt wrecked his car today. Question: What did Matt wreck today? ", return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today?  Answer: his car Context: Matt_Trucks_in_the_United_States_during_the_Great_WarThe first major American naval battle


In [ ]:
# beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today?  Answer: his car Context: Matt_CummingsMatt was born in New Haven, Connecticut. He attended the University of Connecticut, where he earned a degree in


In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=3,
    no_repeat_ngram_size=2, #
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today?  Answer: a Ford F-150 Context: New_York_CityNew York City is home to some of the world's most famous museums, including the Museum of


In [ ]:
# sampling with temperature

tf.random.set_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7 # 1
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: a Ford F-150 Context: Matt_Kirk_Kirk_Kirk_Kirk was born in Dublin, Ireland on 19 October 1918.


In [ ]:
# top k sampling

tf.random.set_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: a Ford X5 Context: Matt_TortonMatt Trenton's body, from 1978 to 1990 had the rear fender set at 18 inches long


In [ ]:
# top p sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: on the platform in a position where he was thrust into one of the rear cars, hit some concrete, had the rear fenders off, bent his car


In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=20,
    temperature=0.7,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: a Ford F-150 Context: Matt_Kirk_Kirk_Kirk_Kirk was born in Atlanta, Georgia on May 18, 1951


In [ ]:
# sevaral variants of generation

tf.random.set_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: a Ford X5 Context: Matt_TortonMatt Trenton's body, from the 1980s until the late '90s, has been described
1: Context: Matt wrecked his car today. Question: What did Matt wreck today? Answer: the car was painted black Context: Matt_Troyer_jr_1914One hundred and seventy-five years after they first met, in
2: Context: Matt wrecked his car today. Question: What did Matt wreck today? ʻDates: 2012, 2009 Context: MattThe "Racetrack" refers to the part of the city's west side that hosts the University.


In [ ]:
# example 2 (prompt with the word "Answer: ")

In [ ]:
input_ids = tokenizer.encode("Context: My name is Jack. Question: What is your name? Answer: ", return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? Answer: _________Jack Context: My_name_is_Jack_My name is Jack. Jack is a fictional character in the popular television series The Simpsons. He is


In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=20,
    temperature=0.7,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? Answer: _____________ Context: My_name_is_Jack_my_name is a combination of my last name, my last name, and the last name of my


In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=3,
    no_repeat_ngram_size=2, #
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? Answer: _____ Context: Jack_of_the_StaircaseJack of the House is a fictional character in the popular television series The Simpsons. He is the son of


In [ ]:
# example 2 (prompt without the word "Answer: ")

In [ ]:
input_ids = tokenizer.encode("Context: My name is Jack. Question: What is your name? ", return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? _________Jack_Cameron Context: My_name_is_Jack_CameronThe first time I met Cameron was in the summer of 2008, when he was in


In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=20,
    temperature=0.7,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? _______________ Context: My_name_is_Jack_my_nameBecause of its popularity, Jack is often used as a nickname in the UK. It has been used


In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=3,
    #no_repeat_ngram_size=2, # with this parameter the model gave meaningless answer
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: My name is Jack. Question: What is your name? _________________My name is Jack. Question: What is your name? _________________My name is Jack. Question: What is your name? _________________My name is


In [ ]:
# example 3 (prompt with the word "Answer: ")

In [ ]:
question = ''
question += 'Question: ' + small_ds['validation'][0]['question']
context = ''
context += 'Context: ' + small_ds['validation'][0]['context']
info = ''
info += context + ' ' + question + 'Answer: '
print(info)

Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50?Answer: 


In [ ]:
input_ids = tokenizer.encode(info, return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=500)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50?Answer: Âthe Denver Broncos Context: Super_B

In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=500,
    top_p=0.92,
    top_k=20,
    temperature=0.7,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50?Answer:  The Broncos Context: Super_Bowl_50_

In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=500,
    num_beams=3,
    no_repeat_ngram_size=2, #
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50?Answer: _________NFL_CONFERENCE Context: Ame

In [ ]:
# example 3 (prompt without the word "Answer: ")

In [ ]:
question = ''
question += 'Question: ' + small_ds['validation'][0]['question']
context = ''
context += 'Context: ' + small_ds['validation'][0]['context']
info = ''
info += context + ' ' + question
print(info)

Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50?


In [ ]:
input_ids = tokenizer.encode(info, return_tensors='tf')

In [ ]:
# greedy search

greedy_output = model.generate(input_ids, max_length=500)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50? Answer: the Denver Broncos Context: Super_B

In [ ]:
# another beam search

beam_output = model.generate(
    input_ids,
    max_length=500,
    num_beams=3,
    no_repeat_ngram_size=2, #
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50? Answer: the New England Patriots Context: A

In [ ]:
# mixed sampling

tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=500,
    top_p=0.92,
    top_k=20,
    temperature=0.7,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Question: Which NFL team represented the AFC at Super Bowl 50? Answer: the National Football League Contex

In [ ]:
# T5

In [ ]:
# default question-answering pipeline (based on encoder models)

from transformers import pipeline

In [ ]:
answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
answerer(question = small_ds['validation'][0]['question'], context = small_ds['validation'][0]['context'])

{'score': 0.6159080266952515,
 'start': 177,
 'end': 191,
 'answer': 'Denver Broncos'}

In [ ]:
# question-answering pipeline is unable to deal with generative models like T5, so
# I can't use saved model

In [ ]:
###### this is why:

In [ ]:
prefix = 'question-answering: '
question = ''
question += 'question: ' + small_ds['validation'][0]['question']
context = ''
context += 'context: ' + small_ds['validation'][0]['context']
info = ''
info += prefix + question + ' ' + context
print(info)

question-answering: question: Which NFL team represented the AFC at Super Bowl 50? context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [ ]:
# saved model attempt 1 (q-a pipeline with generative model -- fail)
answerer = pipeline("question-answering", model = "./t5-small-qa2", tokenizer = AutoTokenizer.from_pretrained("t5-small"))

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
The model 'T5ForConditionalGeneration' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuesti

In [ ]:
answerer(question = small_ds['validation'][0]['question'], context = small_ds['validation'][0]['context'])

ValueError: ignored

In [ ]:
answerer(info)

ValueError: ignored

In [ ]:
# saved model attempt 2 (generative pipeline, q-a prompt -- fail)
answerer = pipeline("text2text-generation", model = "./t5-small-qa2", tokenizer = AutoTokenizer.from_pretrained("t5-small"))

In [ ]:
answerer(question = small_ds['validation'][0]['question'], context = small_ds['validation'][0]['context'])

TypeError: ignored

In [ ]:
######

In [ ]:
# default text2text-generation pipeline

answerer = pipeline("text2text-generation")

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
prefix = 'question-answering: ' # works as well with prefix = 'text2text-generation: '
question = ''
question += 'question: ' + small_ds['validation'][0]['question']
context = ''
context += 'context: ' + small_ds['validation'][0]['context']
info = ''
info += prefix + question + ' ' + context
print(info)

question-answering: question: Which NFL team represented the AFC at Super Bowl 50? context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [ ]:
answerer(info)

[{'generated_text': 'Denver Broncos'}]

In [ ]:
# saved model text2text-generation pipeline

answerer = pipeline("text2text-generation", model = "./t5-small-qa2", tokenizer = AutoTokenizer.from_pretrained("t5-small"))

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
answerer(info)

[{'generated_text': 'Denver Broncos'}]

In [ ]:
# generating answers

model_inputs = tokenizer(info,  max_length=1024, padding='max_length', truncation=True)

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

In [ ]:
raw_pred

array([[    0, 12154,  4027,    29,   509,     7,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]])

In [ ]:
tokenizer.decode(raw_pred[0])

'<pad> Denver Broncos</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
# experiments with generating

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./t5-small-qa2") #, from_pt=True, pad_token_id=tokenizer.eos_token_id)


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# example 1

In [ ]:
# greedy output

input_ids = tokenizer("question-answering: question: What is wonderful? context: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The house


In [ ]:
# activate beam search and early_stopping
input_ids = tokenizer("question-answering: question: What is wonderful? context: The house is wonderful.", return_tensors="pt").input_ids
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

The house


In [ ]:
# top k and top p sampling with temperature

tf.random.set_seed(0)

input_ids = tokenizer(info, return_tensors="pt").input_ids
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    temperature=0.7,
    top_p=0.92,
    top_k=2
)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

In [ ]:
# example 2

In [ ]:
q = "question-answering: question: What is your name? context: My girl's name is Ester Exposito, my name is Roberto Claros de Bunos Silvia and my friend's name is Tom Holland."

In [ ]:
# greedy output

input_ids = tokenizer(q, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Ester Exposito


In [ ]:
# activate beam search and early_stopping
input_ids = tokenizer(q, return_tensors="pt").input_ids
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Ester Exposito


In [ ]:
# top k sampling

tf.random.set_seed(0)

input_ids = tokenizer(q, return_tensors="pt").input_ids
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=2
)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Ester Exposito


In [ ]:
# top k and top p sampling

tf.random.set_seed(0)

input_ids = tokenizer(q, return_tensors="pt").input_ids
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    temperature=0.7,
    top_p=0.92,
    top_k=2
)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Ester Exposito


In [ ]:
# mixed sampling with sevaral number of sequences

tf.random.set_seed(0)

input_ids = tokenizer(info, return_tensors="pt").input_ids

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

In [ ]:
# example 3

In [ ]:
prefix = 'question-answering: '
question = ''
question += 'question: ' + small_ds['validation'][0]['question']
context = ''
context += 'context: ' + small_ds['validation'][0]['context']
info = ''
info += prefix + question + ' ' + context
print(info)

question-answering: question: Which NFL team represented the AFC at Super Bowl 50? context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [ ]:
# experiments with generating

from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./t5-small-qa2") #, from_pt=True, pad_token_id=tokenizer.eos_token_id)

# greedy output

input_ids = tokenizer(info, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Denver Broncos


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# activate beam search and early_stopping
input_ids = tokenizer(info, return_tensors="pt").input_ids
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

American Football Conference


In [ ]:
# top k sampling

tf.random.set_seed(0)

input_ids = tokenizer(info, return_tensors="pt").input_ids
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=2
)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Denver Broncos


In [ ]:
# top k and top p sampling with temperature

tf.random.set_seed(0)

input_ids = tokenizer(info, return_tensors="pt").input_ids
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    temperature=0.7,
    top_p=0.92,
    top_k=2
)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Denver Broncos


In [ ]:
# mixed top k and top p sampling with several sequences

tf.random.set_seed(0)

input_ids = tokenizer(info, return_tensors="pt").input_ids

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: Denver Broncos
1: Denver Broncos
2: Denver Broncos


## Отчет

Напишите, что попробовали сделать, какие модели использовали, с чем экспериментировали, проанализируйте результаты.

(TODO: describe results)

ОТЧЕТ: задачей было научить модели отвечать на вопросы по контексту, используемые модели: gpt2 и t5-small (генеративные модели на основе 1) декодера и 2) энкодера-декодера). Можно было бы аналогично попробовать другие модели тех же групп (gpt и t5). Затем я обучала модель на основе энкодера distilbert-base-uncased предсказывать спаны начала и конца ответа в контексте -- об этом в разделе бонус.

В gpt модели я экспериментировала с форматами промпта, однако получился только один (текстовые файлы, где данные подаются в формате 'Context: Question: Answer: '). В t5 модели я экспериментировала с префиксом задания ('question-answering' или 'text2text-generation'), а также с тем, в каком порядке подавать question и context. Затем в пайплайне и инференсе модели я сравнивала дефолтный пайплайн с пайплайном сохраненной модели, а также экспериментировала с параметрами генерации ответа - методами декодирования в обеих моделях на нескольких примерах. (Еще можно было бы попробовать давать t5 модели контекст и вопрос в виде text и text_pair для токенайзера).

Результаты и анализ:

T5: хуже всего качество (выше лосс и ниже meteor) получается если промпт в формате 'context: question: ' в таком порядке (префикс question-answering). Примерно одинаковое качество, если промпт в формате 'question: context: ', при обоих префиксах question-answering и text2text-generation (однако на text2text-generation чуть лучше -- чуть ниже лосс и выше meteor). Возможно, это связано с тем, что исходые модели уже немного предобучены задаче question-answering именно с таким форматом промпта (это видно и в пайплайне в порядке подачи инпута). Пайплайны t5 в целом очень неплохо справляются с ответами на вопросы, как дефолтные, так и с сохраненной моделью. Изменения параметов генерации при декодировании практически не влияли на ответы (кроме одного случая в примере 3 -- early stopping + beam search, где вместо правильного ответа 'Denver Broncos' модель выдала другое. В примере 2, какими бы ни были параметры генерации, модель выдавала одинаковый неправильный ответ (пример, где в контексте перечисляются имена девушки, "меня" и другие, а потом спрашивается 'what is your name?'. Модель всегда отвечается первое имя, то есть имя девушки.

GPT2: получился только один формат промпта, качество в среднем хуже, чем у t5 (возможно, потому что у t5 есть лейблы-таргет, а gpt нужно больше обучать и файн-тьюнить для этого (или иначе обозначать таргет, например, маскированием). Что касается пайплайна и инференса, то основной проблемой было то, что модель после генерации ответа продолжала генерировать сама новый контекст и тд (можно было бы уменьшить допустимую длину генерации). Также я экспериментировала с промптом для пайплайна: 'Context: ... Question: ... Answer: ' или 'Context: ... Question: ... '. Второй вариант дал значительно более осмысленные результаты (иногда модель отвечала почти правильно), а в первом варианте чаще всего выдавала бессмысленные символы. С параметрами генерации при декодировании были гораздо более разнообразые результаты, чем у t5. Общие наблюдения: для коротких промптов лучше подходит жадный поиск и beam поиск (часто правильные ответы). При sampling, top-k, top-p, temperature и их комбинациях на коротких промптах не очень результаты. На длинных промптах (пример 3, напр), наоборот, greedy и beam поиск хуже, а sampling, top-k, top-p, temperature и их комбинации лучше и даже выдают иногда логически правильные ответы.


В целом точность не очень высокая из-за ограничений: лимит на объем cuda и ОЗУ, лимит на GPU, маленькая мощность. Поэтому невозможно обучать модель большее время, на большем количестве данных, больше эпох, делать меньше размер батчей и тд.

Что можно было бы улучшить: добавить в gpt модель подсчет метрик, таких же, какие использовались для t5 (rouge, meteor...). Добавить в gpt маскирование части с ответом нулями, а частей с контекстом и вопросом единицами (попытка производилась, но не была доведена до конца). Кроме того, можно было бы еще подумать о том, как более ясно передавать модели gpt задачу именно ответов на вопросы по контексту. Также (при отсутствии ограничений) уменьшение размера батчей, увеличение размера датасета при обучении, увеличение количества эпох и времени обучения, добавление слоев и loss'ов, fine-tuning и тд дали бы лучшие результаты у обеих моделей gpt2 и t5-small.

## Бонус

В датасете SQUAD ответы на вопросы содержатся внутри контекста. Можно попробовать обучить модель на основе энкодера для предсказания начала и окончания ответа по токенам контекста:
![](https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-1-4842-6664-9_5/MediaObjects/498208_1_En_5_Fig2_HTML.jpg)

In [ ]:
# variant 1

In [ ]:
# using AutoModelForQuestionAnswering from pretrained distilbert model

model_checkpoint = "distilbert-base-uncased"

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
# preprocess function returning model inputs including start and end tokens of answer

# эта функция имеет 2 недочета: она возвращает start и/или end токены -100, если
# не нашла токенизированный ответ в контексте тк 1) он токенизировался не так, как в контексте, но это редко и
# 2) truncation обрезала кусок с ответом - тоже довольно редко, только в длинных примерах

max_input_length = 512

def indexation(examples):
    input_question = [doc for doc in examples['question']]
    input_context = [doct for doct in examples['context']]
    model_inputs = tokenizer(text = input_question, text_pair = input_context, max_length=max_input_length, truncation = True, padding='max_length')

    start_tokens = []
    end_tokens = []
    for i in range(len(examples['id'])):
        start_pos = examples['answers'][i]['answer_start'][0]
        leng = len(examples['answers'][i]['text'][0])
        end_pos = start_pos+leng
        tok = tokenizer(examples['context'][i][start_pos:end_pos], add_special_tokens = False)
        if tok["input_ids"][0] in model_inputs["input_ids"][i] and tok["input_ids"][-1] in model_inputs["input_ids"][i]:
            start_tokens.append(model_inputs["input_ids"][i].index(tok["input_ids"][0]))
            end_tokens.append(model_inputs["input_ids"][i].index(tok["input_ids"][-1]))
        else:
            start_tokens.append(-100)
            end_tokens.append(-100)

    model_inputs["start_positions"] = start_tokens
    model_inputs["end_positions"] = end_tokens

    return model_inputs

In [ ]:
tokenized_ds = small_ds.map(indexation, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
training_args = TrainingArguments(
    output_dir="./qa_distil-bert",
    overwrite_output_dir=True,
    learning_rate=4e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  #1
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=20, #200
    save_strategy="steps",
    save_steps=20, #200
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
    #compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
20,No log,4.338180
40,No log,3.863014
60,No log,3.672251
80,No log,3.410738
100,No log,2.954583
120,No log,2.687356
140,No log,2.530325
160,No log,2.441576
180,No log,2.367413
200,No log,2.327408


TrainOutput(global_step=274, training_loss=2.9947768162636863, metrics={'train_runtime': 575.6685, 'train_samples_per_second': 15.217, 'train_steps_per_second': 0.476, 'total_flos': 1144521156280320.0, 'train_loss': 2.9947768162636863, 'epoch': 2.0})

In [ ]:
# creating test for evaluation

test_ds = ds['train'].filter(lambda example, index: index % 50 == 0, with_indices=True)

In [ ]:
test_tokenized = test_ds.map(indexation, batched=True)

In [ ]:
trainer.evaluate(eval_dataset=test_tokenized)

{'eval_loss': 2.101128578186035,
 'eval_runtime': 28.7067,
 'eval_samples_per_second': 61.031,
 'eval_steps_per_second': 3.832,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

In [ ]:
# pipeline for our saved model

from transformers import pipeline
question_answerer = pipeline('question-answering', model="./qa_distil-bert", tokenizer = tokenizer)

context = 'Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.'

result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'SQuAD', score: 0.305, start: 146, end: 151


In [ ]:
# pipeline, inference for our saved model

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AutoModelForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = AutoModelForQuestionAnswering.from_pretrained("./qa_distil-bert")

question, context = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))
print(answer_start_index)

jim henson
tensor(3)


In [ ]:
# pipeline, inference for default model (DistilBertForQuestionAnswering)

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))
print(answer_start_index)

a nice puppet
tensor(10)


In [ ]:
# pipeline for default model

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

context = 'Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.'

result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'SQuAD dataset', score: 0.4704, start: 146, end: 159


ВЫВОДЫ: качество получилось довольно плохое, но заметно, что при обучении loss стабильно падал, поэтому, вероятно, если бы была возможность (хватало бы мощности, памяти и не было бы ограничений на использование ускорителя GPU) обучать модель дольше, на большем количестве данных, с большим количеством эпох, меньшим размером батчей и тд, то качество было бы лучше. Также можно было бы доработать небольшие недочеты функции обработки (см выше). Тем не менее, loss на тестовой выборке был меньше, чем при обучении, а при использовании пайплайна сохраненной модели она правильно предсказала начало ответа (по сравнению с дефолтной - см дефолтный пайплайн), что уже результат. Однако в целом она нуждается в гораздо большем размере обучения, чтобы выдавать лучшую точность.

In [ ]:
# variant 2

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_datasets = small_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [ ]:
training_args = TrainingArguments(
    output_dir="./qa_distil-bert2",
    overwrite_output_dir=True,
    learning_rate=4e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  #1
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=20, #200
    save_strategy="steps",
    save_steps=20, #200
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
20,No log,4.317134
40,No log,3.812629
60,No log,3.606638
80,No log,3.244871
100,No log,2.642961
120,No log,2.336683
140,No log,2.160102
160,No log,2.039011
180,No log,1.973171
200,No log,1.907295


TrainOutput(global_step=274, training_loss=2.7405992494012317, metrics={'train_runtime': 444.2106, 'train_samples_per_second': 19.72, 'train_steps_per_second': 0.617, 'total_flos': 858390867210240.0, 'train_loss': 2.7405992494012317, 'epoch': 2.0})

In [ ]:
# creating test for evaluation

test_ds = ds['train'].filter(lambda example, index: index % 50 == 0, with_indices=True)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [ ]:
test_tokenized = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/1752 [00:00<?, ? examples/s]

In [ ]:
trainer.evaluate(eval_dataset=test_tokenized)

{'eval_loss': 1.705405831336975,
 'eval_runtime': 24.3052,
 'eval_samples_per_second': 72.083,
 'eval_steps_per_second': 4.526,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

In [ ]:
# pipeline for our saved model

from transformers import pipeline
question_answerer = pipeline('question-answering', model="./qa_distil-bert2", tokenizer = tokenizer)

context = 'Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.'

result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the SQuAD dataset', score: 0.1561, start: 142, end: 159


In [ ]:
# pipeline, inference for our saved model

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AutoModelForQuestionAnswering
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = AutoModelForQuestionAnswering.from_pretrained("./qa_distil-bert2")

question, context = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))
print(answer_start_index)

jim henson was a nice puppet
tensor(7)


ВЫВОДЫ: во втором варианте немного другая функция препроцессинга, практически без недочетов первой, и качество на том же размере обучения и данных лучше. Например, во втором пайплайне уже более-менее правильный логически ответ.